### Предварительная обработка текста сообщений в ЗНО

In [ ]:
# функция удаления имен (список имен берется из файла с именами names.csv)

names = pd.read_csv('names.csv')[['name', 'type', 'sex']]

def remove_names(string, names=names):
    string = ' '+string.lower()+' '
    user_name_template = '_user_name_'
    type_order = ['f','o','i']
    for t in type_order:
        all_names = names[names['type']==t]['name'].tolist()
        all_types = names[names['type']==t]['type'].tolist()
        all_sex = names[names['type']==t]['sex'].tolist()

        for i in range(len(all_names)):
            name_ = all_names[i]
            type_ = all_types[i]
            sex_ = all_sex[i]

            arr = [name_]
            if type_ == 'i' and sex_ == 'm':
                if name_[:-2] in string:
                    if name_[-2:] in ['ай', 'ей', 'ий']:
                        string = re.sub(' '+name_[:-2]+'\w*вна ', ' '+user_name_template+' ', string)
                        string = re.sub(' '+name_[:-2]+'\w*вич ', ' '+user_name_template+' ', string)
                        #arr.extend( re.findall(' '+name_[:-2]+'\w*вна ', string))
                        #arr.extend( re.findall(' '+name_[:-2]+'\w*вич ', string))
                    if name_[-2:] in ['ил']: 
                        string = re.sub(' '+name_[:-2]+'йл\w* ', ' '+user_name_template+' ', string)
                        string = re.sub(' '+name_[:-2]+'\w*вна ', ' '+user_name_template+' ', string)
                        string = re.sub(' '+name_[:-2]+'\w*вич ', ' '+user_name_template+' ', string)
                        #arr.extend( re.findall(' '+name_[:-2]+'йл\w* ', string))
                        #arr.extend( re.findall(' '+name_[:-2]+'\w*вна ', string))
                        #arr.extend( re.findall(' '+name_[:-2]+'\w*вич ', string))
            if len(name_)>=5:
                string = re.sub(' '+name_+'\w* ', ' '+user_name_template+' ', string)
                #arr.extend( re.findall(' '+name_+'\w* ', string))
            if len(name_)>2 and name_ in string:
                string = string.replace(' '+name_+' ',' '+user_name_template+' ')

            arr = np.unique(arr)
            if len(arr)>1 and 1==0:
                for name_val in arr:
                    string = string.replace(name_val,' '+user_name_template+' ')
                    
    string = re.sub(' ('+user_name_template+'\s*)+ ', ' '+user_name_template+' ', string)
    return string

# функция неразбиения слов с частицами "не", "ни","без" т.к. это может быть значимо при анализе информации в ЗНО
def collapse_ne(string):
    string = re.sub(r'\s+', '  ', string)
    string = re.sub(r' не\s+(\w+)', r' не\1', string)
    string = re.sub(r' ни\s+(\w+)', r' не\1', string)
    string = re.sub(r' без\s+(\w+)', r' без\1', string)
    return string

# фукция сворачивания массива в строку и преобразование в нижний регистр
def to_one_string(arr, is_lower=True):
    delimeter = list(set([chr(i).lower() for i in range(255)]) -set(' '.join(arr).lower()) )[-1]
    if is_lower:
        return delimeter, (' '+(' '+delimeter+' ').join(arr)+' ').lower()
    return delimeter, ' '+(' '+delimeter+' ').join(arr)+' '

In [ ]:
# нормализация текста при помощи MyStem
import threading
from pymystem3 import Mystem
from math import floor
semaphore = threading.BoundedSemaphore()
m = Mystem()

def toMystem(text):
#  составление масива слов 
    arr = text.split()
    delimeter, uniq_word_text = to_one_string(list(set(arr)))
    print('to_one_string finished. uniq word count is ', len(uniq_word_text.split(delimeter)),' word count is ',(len(arr)))
    dict_arr = {}
    for i, word in enumerate(arr):
        if word not in dict_arr:
            dict_arr[word] = []
        dict_arr[word].append(i)
        
# нормализация слов                                              
    semaphore.acquire() # decrements the counter
    lemmas = m.lemmatize(uniq_word_text)
    semaphore.release() # increments the counter
    print('m.lemmatize finished')

# составление нового массива слов (уже нормализованных)                                              
    old_word = uniq_word_text.split(delimeter)
    new_word = remove_names(' '.join(lemmas)).split(delimeter)
    print('remove_names finished')
    new_arr = ['' for i in range(len(arr))]
    for i, word in enumerate(old_word):
        if i/10000 == floor(i/10000):
            print(i)
        for j in dict_arr[word.strip()]:
            new_arr[j] = new_word[i]
        
    text = ' '.join(new_arr)
    text = collapse_ne(text)
    return text

### Подборка наилучших параметров алгоритма в модели (на примере LogisticRegression) при помощи GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]
              , 'class_weight': [None,'balanced']
#              , 'multi_class':['ovr','multinomial']
#              ,'solver': ['liblinear','sag']
             }
#поиск наилучших параметров
grid_search = GridSearchCV(LogisticRegression(random_state=9,n_jobs=-1), param_grid, cv=5)
grid_search.fit(X_train_ve, y_train)

print("Правильность на тестовом наборе: {:.2f}".format(grid_search.score(X_test_ve, y_test)))
print("Наилучшие значения параметров: {}".format(grid_search.best_params_))
print("Наилучшее значение кросс-валидац. правильности: {:.2f}".format(grid_search.best_score_))
print("Наилучшая модель:\n{}".format(grid_search.best_estimator_))

### Удаление запросов на группу, которая не подлежит классификации

In [ ]:
data.at[data['SM_class'] == 'Ф', 'SM_class'] = 16


for (name, series) in data.iterrows():
    if data.at[name,'Группа'].count('ОСРСЦентр')==0: data.at[name,'Группа']=np.nan 

### Транспонирование коэффициентов классификатора по классам

In [ ]:
#clf.coef_ - работает для LogisticRegression
#[1, 2, 3, 4] - список классов, на которые распределяет наш классификатор
dd = pd.DataFrame(data=np.transpose(clf.coef_), index=list(vectorizer.get_feature_names()), columns=[1, 2, 3, 4])

### Класс, описывающий классификатор по принципу "один против всех"

In [ ]:
import xgboost as xgb

class One_vs_all():
    def __init__(self, train, y_train, random_state, params, regressor, vectorizer
                 , num_boost_round = 500):
        self.train = train
        self.y_train = y_train
        self.random_state = random_state
        self.params = params
        self.num_boost_round = num_boost_round
        self.y_dummied = ''
        self.one_vs_all_models = {}
        
        self.vectorizer = vectorizer
        self.regressor = regressor
        self.is_preprocessed = False
        self.cv_dict = {}
    
    def remove_zero_rows(self,matr):
        nonzero_row_indice, _ = matr.nonzero()
        unique_nonzero_indice = np.unique(nonzero_row_indice)
        return matr[unique_nonzero_indice], unique_nonzero_indice

    def preprocess_fit(self):
        if self.is_preprocessed:
            return
        self.vectorizer = self.vectorizer.fit(self.train)
        train_tfidf = self.vectorizer.transform(self.train)
        
        self.train_tfidf, not_zero_rows = self.remove_zero_rows(train_tfidf)
        
        self.y_train= self.y_train.iloc[not_zero_rows]

        self.y_dummied = pd.get_dummies(self.y_train)
        for class_name in self.y_dummied.columns:
            if self.y_dummied[class_name].sum() < 10:
                self.y_dummied.drop(class_name, axis=1, inplace=True)
                
        self.is_preprocessed = True
    
    def model_fit(self):
        for class_name in self.y_dummied.columns:
            y_train = self.y_dummied[class_name]
            temp_params = self.params
            scale = len(y_train)/y_train.sum() 
            if self.regressor == xgb:
                dtrain = xgb.DMatrix(self.train_tfidf, y_train)
                temp_params["scale_pos_weight"] = scale
                class_model = xgb.train(temp_params, dtrain, self.num_boost_round)
            else:
                dtrain = self.train_tfidf
                params = self.params
                params['random_state'] = self.random_state
                class_model = self.regressor(**params).fit(dtrain, y_train)
            
            self.one_vs_all_models[class_name] = class_model
        #del self.train_tfidf
        #del self.y_dummied
    
    def cv_model(self):
        self.preprocess_fit()
        if len(self.cv_dict) > 0:
            return self.cv_dict
        
        cv_dict = {}        
        #sss = StratifiedShuffleSplit(n_splits=5, test_size=0.30)
        for class_name in self.y_dummied.columns:
            print('cv for ',class_name)
            y_train = self.y_dummied[class_name]
            if self.regressor == xgb:
                dtrain = xgb.DMatrix(self.train_tfidf, y_train)
                cv_info = xgboost.cv(self.params, dtrain, num_boost_round=self.num_boost_round, nfold=5, stratified=True,metrics='logloss',verbose_eval=100)
                '''else:
                params = self.params
                params['random_state'] = self.random_state
                scoresSSS = cross_val_score(self.regressor(**params), self.train_tfidf, y_train, cv=5, n_jobs=-1)
                cv_info = np.average(scoresSSS)
                #print(cv_info)'''
            cv_dict[class_name] = cv_info
        #del self.train_tfidf
        #del self.y_dummied
        self.cv_dict = cv_dict
        return cv_dict
    
    def clear(self):
        del self.train_tfidf
        del self.y_dummied
        
    def predict_single(self, text):
        test = [text]
        test_tfidf = self.vectorizer.transform(test)
        if self.regressor == xgb:
            dtest = xgb.DMatrix(test_tfidf)
        else:
            dtest = test_tfidf
        result = {}
        if type(self.regressor).__name__ in ['type', 'module']:
            for class_name in self.one_vs_all_models:
                model = self.one_vs_all_models[class_name]
                proba = model.predict(dtest)[0].apply(str)
                if proba > 0.5:
                    result[class_name] = proba
        else:
            result['result'] = str(self.regressor.predict(dtest)[0])
        return result
    
    def predict(self, test):
        #print(type(test).__name__, test)
        if isinstance(test, str):
            return self.predict_single(test)
            
        test_tfidf = self.vectorizer.transform(test)
        if self.regressor == xgb:
            dtest = xgb.DMatrix(test_tfidf)
        
        if isinstance(test, pd.Series):
            index = test.index
            result = pd.DataFrame(columns = self.one_vs_all_models.keys(), index=index)
        else:
            result = pd.DataFrame(columns = self.one_vs_all_models.keys())
        #result = pd.DataFrame()
        
        if type(self.regressor).__name__ in ['type', 'module']:
            for class_name in self.one_vs_all_models:
                model = self.one_vs_all_models[class_name]
                if self.regressor == xgb:
                    proba = model.predict(dtest)
                else:
                    proba = model.predict(test_tfidf)
                result[class_name.apply(str)] = proba.apply(str)
            return result.idxmax(axis=1).apply(str), result.apply(str)
        else:
            result['result'] = self.regressor.predict(test_tfidf)
            return result.max(axis=1).apply(str), result.apply(str)

In [ ]:
# формирование модели по принципу "один против всех"
clf_model = One_vs_all(train=None, y_train=None, random_state=42, params={}, regressor=clf, vectorizer=vectorizer)

# предсказание данных 
y_pred, result=clf_model.predict(X_test)
# вывод процента точности
print("Правильность на тестовом наборе: {:.2f}".format(np.mean(y_pred == y_test)))